In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-24"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
16427,2024-03-24,Espanha Acb,13:00,Bilbao,Baskonia,2.29,1.63,166.5,1.87,1.89,2.5,1.94,1.38,EVvGhsts,0.436681,0.613497,0.534759,0.529101,0.050178,0.6,0.6,NaN,NaN,200.218,58.883410,0.294096,151.298,52.148651,0.344675,146.220,158.942,186.76,192.66,0.0,0.0,0.0,0.0,0.238107,0.007522,0.238542,2.19,0.438,2.945205,0.000000,0.0,0.000000,-1.94,-0.388,-1.623711,0.000000,0.0,0.000000
16428,2024-03-24,Espanha Acb,13:00,Valencia,Murcia,1.47,2.71,162.5,1.93,1.86,-4.5,1.92,3.90,zDD9Gcdt,0.680272,0.369004,0.518135,0.537634,0.049276,0.6,0.8,NaN,NaN,158.362,82.275365,0.519540,186.226,71.123059,0.381918,105.168,195.614,109.62,192.72,0.0,0.0,0.0,0.0,0.419527,0.026120,0.481124,-2.61,-0.522,-0.900383,0.000000,0.0,0.000000,-1.12,-0.224,-7.633929,0.000000,0.0,0.000000
16429,2024-03-24,Eua Nba,20:00,Milwaukee Bucks,Oklahoma City Thunder,1.74,2.16,232.5,1.81,1.95,-3.5,2.01,2.48,dC197KFP,0.574713,0.462963,0.552486,0.512821,0.037676,0.4,0.2,NaN,NaN,199.498,42.794325,0.214510,188.052,36.108845,0.192015,184.796,194.234,141.45,188.24,0.0,0.0,0.0,0.0,0.152300,0.052657,0.148036,1.65,0.330,2.242424,0.680507,0.7,0.019493,-1.76,-0.352,-3.295455,0.612563,0.6,-0.012563
16430,2024-03-24,Eua Nba,20:00,Minnesota Timberwolves,Golden State Warriors,1.73,2.18,217.5,1.93,1.93,-3.5,2.04,2.60,bLmy3vOt,0.578035,0.458716,0.518135,0.518135,0.036750,0.4,0.2,NaN,NaN,197.658,34.425710,0.174168,176.726,42.945415,0.243006,175.478,275.862,253.12,205.20,0.0,0.0,0.0,0.0,0.162761,0.000000,0.170681,-3.68,-0.736,-0.991848,0.666068,0.3,-0.366068,5.63,1.126,1.047957,0.504175,0.7,0.195825
16431,2024-03-24,Eua Nba,23:00,Los Angeles Lakers,Indiana Pacers,1.69,2.25,240.5,1.85,1.93,-3.5,1.93,2.51,Gdrt2bwm,0.591716,0.444444,0.540541,0.518135,0.036160,0.6,0.2,NaN,NaN,241.780,42.188429,0.174491,227.776,32.572936,0.143004,240.972,186.646,293.97,203.13,0.0,0.0,0.0,0.0,0.201005,0.029930,0.184740,2.92,0.584,1.181507,0.600207,0.7,0.099793,-0.39,-0.078,-16.025641,0.523775,0.4,-0.123775
16432,2024-03-24,Itália Liga A,12:30,Cremona,Venezia,2.40,1.58,155.5,1.85,1.93,2.5,2.04,1.33,GYerF7ob,0.416667,0.632911,0.540541,0.518135,0.049578,0.4,0.4,NaN,NaN,173.460,40.193111,0.231714,134.480,34.651981,0.257674,144.398,129.354,157.50,150.10,0.0,0.0,0.0,0.0,0.291371,0.029930,0.297950,1.92,0.384,3.645833,0.000000,0.0,0.000000,-0.22,-0.044,-13.181818,0.551070,0.5,-0.051070
16433,2024-03-24,Itália Liga A,14:15,Pesaro,Reggiana,2.04,1.78,162.5,1.83,1.95,1.5,1.89,1.65,Y96wGmWi,0.490196,0.561798,0.546448,0.512821,0.051994,0.2,0.4,NaN,NaN,244.570,68.333917,0.279404,173.480,77.863949,0.448835,166.938,174.146,124.02,108.99,0.0,0.0,0.0,0.0,0.096255,0.044896,0.095879,-5.00,-1.000,-1.040000,0.000000,0.0,0.000000,-5.00,-1.000,-0.780000,0.000000,0.0,0.000000
16434,2024-03-24,Itália Liga A,14:30,Scafati,Brindisi,1.51,2.58,160.5,1.80,1.96,-4.5,1.90,3.75,2kbfC5VG,0.662252,0.387597,0.555556,0.510204,0.049849,1.0,0.6,NaN,NaN,224.680,162.569813,0.723562,258.442,210.696205,0.815255,161.396,340.514,203.98,632.50,0.0,1.0,0.0,0.0,0.369978,0.060179,0.463061,1.88,0.376,1.356383,0.528024,0.7,0.171976,-2.62,-0.524,-3.015267,0.000000,0.0,0.000000
16435,2024-03-24,Itália Liga A,15:00,Milano,Basket Napoli,1.18,5.05,158.5,1.83,1.97,-10.5,1.85,10.50,fDWsEtJf,0.847458,0.198020,0.546448,0.507614,0.045477,0.4,0.8,NaN,NaN,110.602,37.499876,0.339052,192.302,91.824518,0.477502,96.368,237.918,71.76,172.64,0.0,0.0,0.0,0.0,0.878492,0.052103,0.990522,1.17,0.234,0.769231,0.000000,0.0,0.000

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,13:00,Espanha Acb,Valencia,Murcia,162.5,1.93,1.0000,Over
1,20:00,Eua Nba,Milwaukee Bucks,Oklahoma City Thunder,232.5,1.81,1.0000,Over
2,20:00,Eua Nba,Minnesota Timberwolves,Golden State Warriors,217.5,1.93,1.0000,Over
3,23:00,Eua Nba,Los Angeles Lakers,Indiana Pacers,240.5,1.85,1.0000,Over
4,14:30,Itália Liga A,Scafati,Brindisi,160.5,1.80,1.0000,Over
5,15:00,Itália Liga A,Milano,Basket Napoli,158.5,1.83,1.0000,Over
6,13:00,Alemanha Bbl,Chemnitz,Alba Berlin,168.5,1.87,1.0000,Over
7,01:15,Austrália Nbl1 Central,Norwood Flames,North Adelaide Rockets,174.5,1.86,1.0000,Over
8,13:30,Áustria Superliga,Traiskirchen,Wels,144.5,1.81,1.0000,Over
9,13:30,Áustria Superliga,Vienna,Oberwart,163.5,1.83,0.9998,Over
